In [14]:
from keras.applications import vgg16
from keras.models import Sequential
from keras.layers import Dense,Input,Flatten,Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import cv2
import numpy as np

In [15]:
# def normalise(image):
#     min_pixel = np.min(image)
#     max_pixel = np.max(image)

#     normalised_image = (image - min_pixel)/(max_pixel - min_pixel)

#     return normalised_image

In [16]:
# def median_filtering(image):
#     image = np.uint8(image)
#     dst = cv2.medianBlur(image,7)
#     return dst

Main Function

In [17]:
# def func(image):
#     dst = median_filtering(image)
#     return normalise(dst)

In [18]:
def func(image):
    # dst = median_filtering(image)
    # return normalise(dst)
    return image

In [19]:
train_path = "/kaggle/input/version1/train_cropped"
test_path = "E:\Cars_datasets\\v1\\test"
val_path = "/kaggle/input/version1/val_cropped"

In [20]:
train_batches = ImageDataGenerator(preprocessing_function=func,rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest').flow_from_directory(directory=train_path,target_size=(224,224),classes=['AlfaRomeo', 'AstonMartin', 'Audi', 'Bentley', 'BMW', 'Citroen', 'Cupra', 'Dacia', 'Daewoo', 'Dodge', 'Fiat', 'Ford', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Kia', 'LandRover', 'Lexus', 'Maserati', 'Mazda', 'MercedesBenz', 'MG', 'MiniCooper', 'Mitsubishi', 'Nissan', 'Peugeot', 'Porche', 'Renault', 'Seat', 'Skoda', 'Smart', 'Subaru', 'Suzuki', 'Tesla', 'Toyota', 'Vauxhall', 'Volvo', 'VW'],batch_size=32)

test_batches = ImageDataGenerator(preprocessing_function=func,    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest').flow_from_directory(directory=test_path,target_size=(224,224),classes=['AlfaRomeo', 'AstonMartin', 'Audi', 'Bentley', 'BMW', 'Citroen', 'Cupra', 'Dacia', 'Daewoo', 'Dodge', 'Fiat', 'Ford', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Kia', 'LandRover', 'Lexus', 'Maserati', 'Mazda', 'MercedesBenz', 'MG', 'MiniCooper', 'Mitsubishi', 'Nissan', 'Peugeot', 'Porche', 'Renault', 'Seat', 'Skoda', 'Smart', 'Subaru', 'Suzuki', 'Tesla', 'Toyota', 'Vauxhall', 'Volvo', 'VW'],batch_size=32, shuffle=False)

val_batches = ImageDataGenerator(preprocessing_function=func).flow_from_directory(directory=val_path,target_size=(224,224),classes=['AlfaRomeo', 'AstonMartin', 'Audi', 'Bentley', 'BMW', 'Citroen', 'Cupra', 'Dacia', 'Daewoo', 'Dodge', 'Fiat', 'Ford', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Kia', 'LandRover', 'Lexus', 'Maserati', 'Mazda', 'MercedesBenz', 'MG', 'MiniCooper', 'Mitsubishi', 'Nissan', 'Peugeot', 'Porche', 'Renault', 'Seat', 'Skoda', 'Smart', 'Subaru', 'Suzuki', 'Tesla', 'Toyota', 'Vauxhall', 'Volvo', 'VW'],batch_size=32)

Found 13566 images belonging to 39 classes.
Found 0 images belonging to 39 classes.
Found 2895 images belonging to 39 classes.


In [21]:
# train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path,target_size=(224,224),classes=['AlfaRomeo', 'AstonMartin', 'Audi'],batch_size=10)

# test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path,target_size=(224,224),classes=['AlfaRomeo', 'AstonMartin', 'Audi'],batch_size=10, shuffle=False)

# val_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=val_path,target_size=(224,224),classes=['AlfaRomeo', 'AstonMartin', 'Audi'],batch_size=10)

In [22]:
vgg16_model = vgg16.VGG16()
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [23]:
for layers in vgg16_model.layers[1:-1]:
    print(layers)

In [24]:
model = Sequential()
model.add(Input(shape=(224, 224, 3)))
for layers in vgg16_model.layers[1:-1]:
    layers.trainable = True
    model.add(layers)
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=39, activation='softmax'))

In [25]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint_callback = ModelCheckpoint("cars_vgg16v2.h5", save_best_only=True)
early_stopping_callback = EarlyStopping(patience=10)
tensorboard_callback = TensorBoard(log_dir="E:\log_tf")
model.fit(x=train_batches,validation_data=val_batches,epochs=100, verbose=1, callbacks=[checkpoint_callback,tensorboard_callback])

Epoch 1/100
424/424 [==============================] - 379s 891ms/step - loss: 3.6625 - accuracy: 0.0374 - val_loss: 3.6271 - val_accuracy: 0.0380
Epoch 2/100
424/424 [==============================] - 295s 694ms/step - loss: 3.6022 - accuracy: 0.0500 - val_loss: 3.4691 - val_accuracy: 0.0801
Epoch 3/100
424/424 [==============================] - 311s 734ms/step - loss: 3.4653 - accuracy: 0.0817 - val_loss: 3.2837 - val_accuracy: 0.1157
Epoch 4/100
424/424 [==============================] - 290s 683ms/step - loss: 3.1957 - accuracy: 0.1373 - val_loss: 2.8670 - val_accuracy: 0.2097
Epoch 5/100
424/424 [==============================] - 295s 696ms/step - loss: 2.8865 - accuracy: 0.2065 - val_loss: 2.5761 - val_accuracy: 0.2746
Epoch 6/100
424/424 [==============================] - 290s 684ms/step - loss: 2.5257 - accuracy: 0.2976 - val_loss: 2.2586 - val_accuracy: 0.3668
Epoch 7/100
424/424 [==============================] - 292s 689ms/step - loss: 2.1994 - accuracy: 0.3769 - val_loss: 1